In [0]:
from azure.storage.fileshare import ShareServiceClient
import os
import tqdm

In [0]:
df = spark.sql("""
SELECT *
FROM 1_inland.sectra.pacs_file_copy
WHERE active_ind = 1 AND LOWER(status) != 'completed'              
""")

In [0]:
display(df)

In [0]:
acc_name = dbutils.secrets.get(scope = "adc_store", key = "pacs_intfileshare_accname")
acc_key = dbutils.secrets.get(scope = "adc_store", key = "pacs_intfileshare_acckey")

# Connection string
connection_string = f"DefaultEndpointsProtocol=https;AccountName={acc_name};AccountKey={acc_key};EndpointSuffix=core.windows.net"

# File share name
share_name = "intfileshare"

# Get a share client via connection string
share_client = ShareServiceClient.from_connection_string(connection_string).get_share_client(share_name)

In [0]:
# TODO: make a UDF to move file from src to dst and return status